In [134]:
# importamos la libreria
import tensorflow as tf
from tensorflow.python import control_flow_ops
import time, os

# importamos librerías adicionales
import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob
from skimage.color import rgb2gray
import pathlib
import skimage.data as imd
from sklearn.model_selection import train_test_split
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import to_categorical, shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.metrics import Accuracy
from tflearn import DNN
import tflearn
tf.reset_default_graph()

%matplotlib inline

# funciones de ayuda
def plot_img(im):
    im_grey = rgb2gray(im)
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    plt.imshow(im)
    plt.axis('off')
    plt.title('Original Image')

    plt.subplot(1,2,2)
    plt.imshow(im_gray, cmap='gray')
    plt.axis('off')
    plt.title('Grayscale Image')

    plt.tight_layout()
    plt.show()

def load_data(path):
    images, labels = [], []
    folders = os.listdir(path)
    for i in range(len(folders)):
        dir_img = os.path.join(path, folders[i])
        for j in os.listdir(dir_img):
            img = os.path.join(dir_img, j)
            img = imd.imread(img)
            try:
                labels.append(i)
                images.append(np.array(img))
            except:
                continue
    return np.array(images, dtype=np.float64), np.array(labels)

In [135]:
images, labels = load_data("./data")
images.dtype

dtype('float64')

In [136]:
""" Process our TF dataset """
# split our tf set in a test and training part
X, X_test, Y, Y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
#X, Y = shuffle(X, Y)
# encode our labels
Y = to_categorical(Y, 5)
Y_test = to_categorical(Y_test, 5)
X.shape

(3368, 32, 32, 3)

In [137]:
tf_img_prep = ImagePreprocessing()
tf_img_prep.add_featurewise_zero_center()
tf_img_prep.add_featurewise_stdnorm()


# Randomly create extra image data by rotating and flipping images
tf_img_aug = ImageAugmentation()
tf_img_aug.add_random_flip_leftright()
tf_img_aug.add_random_rotation(max_angle=30.)


In [138]:
# Convolutional network building
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=tf_img_prep,
                     data_augmentation=tf_img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = dropout(network, 0.5)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=3)
model.fit(X, Y, n_epoch=25, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True)


Training Step: 1324  | total loss: 0.92274 | time: 11.164s
| Adam | epoch: 025 | loss: 0.92274 - acc: 0.6850 -- iter: 3328/3368
Training Step: 1325  | total loss: 0.89822 | time: 12.374s
| Adam | epoch: 025 | loss: 0.89822 - acc: 0.6930 | val_loss: 0.84260 - val_acc: 0.7013 -- iter: 3368/3368
--


In [139]:
import scipy
from skimage.transform import resize
image_path = './daisy.jpg'
img = scipy.ndimage.imread(image_path, mode="RGB")

# Scale it to 32X32
img = scipy.misc.imresize(img, (32, 32),
                                  interp="bicubic").astype(np.float32,
                                  casting='unsafe')
# Predict
predicted = model.predict([img])
print("\n\n\nPrediccion:")
# Check the result.
if np.argmax(predicted[0]) == 0:
    print("\nEs una MARGARITA!")
elif np.argmax(predicted[0]) == 1:
    print("\nEs un DIENTE DE LEON!")
elif np.argmax(predicted[0]) == 2:
    print("\nEs una ROSA!")
elif np.argmax(predicted[0]) == 3:
    print("\nEs un GIRASOL!")
elif np.argmax(predicted[0]) == 4:
    print("\nEs un TULIPAN!")
print("\nProbabilidades: "+str(np.around(predicted[0]*100, decimals=3)))
print("\n\n\n")





Prediccion:

Es un TULIPAN!

Probabilidades: [21.414  0.429  2.796  1.558 73.803]






/home/gosen/.local/share/virtualenvs/src-pbKLSnqF/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  after removing the cwd from sys.path.
/home/gosen/.local/share/virtualenvs/src-pbKLSnqF/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  
